In [3]:
#beginning code from https://www.kaggle.com/shanan93/brain-mri-segmentation-95-5-accuracy
import pandas as pd
import pandas.util.testing as tm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import glob
import random
#%matplotlib inline

In [21]:
main_df = pd.read_csv('archive/lgg-mri-segmentation/kaggle_3m/data.csv')

In [22]:
data_map = []
for sub_dir_path in glob.glob("archive/lgg-mri-segmentation/kaggle_3m/"+"*"):
    try:
        dir_name = sub_dir_path.split('/')[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + '/' + filename
            data_map.extend([dir_name, image_path])
    except Exception as e:
        print(e)


[Errno 20] Not a directory: 'archive/lgg-mri-segmentation/kaggle_3m/data.csv'
[Errno 20] Not a directory: 'archive/lgg-mri-segmentation/kaggle_3m/README.md'


In [23]:
df = pd.DataFrame({"patient_id" : data_map[::2],
                   "path" : data_map[1::2]})
df.head()

,patient_id,path
0,TCGA_CS_6667_20011105,archive/lgg-mri-segmentation/kaggle_3m/TCGA_CS...
1,TCGA_CS_6667_20011105,archive/lgg-mri-segmentation/kaggle_3m/TCGA_CS...
2,TCGA_CS_6667_20011105,archive/lgg-mri-segmentation/kaggle_3m/TCGA_CS...
3,TCGA_CS_6667_20011105,archive/lgg-mri-segmentation/kaggle_3m/TCGA_CS...
4,TCGA_CS_6667_20011105,archive/lgg-mri-segmentation/kaggle_3m/TCGA_CS...


In [31]:
df_imgs = df[~df['path'].str.contains("mask")]
df_masks = df[df['path'].str.contains("mask")]

# File path line length images for later sorting
BASE_LEN = 89 
END_IMG_LEN = 4 
END_MASK_LEN = 9 

# Data sorting
imgs = sorted(df_imgs["path"].values, key=lambda x : int(x[BASE_LEN:-END_IMG_LEN]))
#masks = sorted(df_masks["path"].values, key=lambda x : int(x[BASE_LEN:-END_MASK_LEN]))

ValueError: invalid literal for int() with base 10: ''

In [29]:
# Final dataframe
brain_df = pd.DataFrame({"patient_id": df_imgs.patient_id.values,
                         "image_path": imgs,
                         "mask_path": masks
                        })
def pos_neg_diagnosis(mask_path):
    value = np.max(cv2.imread(mask_path))
    if value > 0 : 
        return 1
    else:
        return 0
    
brain_df['mask'] = brain_df['mask_path'].apply(lambda x: pos_neg_diagnosis(x))
brain_df

NameError: name 'imgs' is not defined